In [5]:
import pandas as pd

# 1. 정답지(Reference) 로드
df_ref = pd.read_csv('data/법정동합본.csv', encoding='euc-kr')
df_main = pd.read_csv('data/real_estate_total_edit.csv', encoding='euc-kr')
df_ref.columns = ['CGG_CD', 'CGG_NM', 'STDG_CD', 'STDG_NM']
# 2. 데이터 타입 통일 (안전한 비교를 위해 모두 문자열로 변환)
# 작업 중인 메인 데이터프레임 이름을 여기에 넣으세요 (예: df_total2)
df_main = df_main.copy() 

df_main.head()

cols_to_str = ['CGG_CD', 'STDG_CD']

for col in cols_to_str:
    df_ref[col] = df_ref[col].astype(str)
    df_main[col] = df_main[col].astype(str)

# ==========================================
# 🔍 1단계: 자치구 검증 (Gu Validation)
# ==========================================

# 정답지에서 {CGG_CD: CGG_NM} 딕셔너리 생성
gu_mapper = dict(zip(df_ref['CGG_CD'], df_ref['CGG_NM']))

# 메인 데이터의 코드에 맞는 '정답 이름'을 매핑
df_main['검증_CGG_NM'] = df_main['CGG_CD'].map(gu_mapper)

# 불일치 찾기 (데이터가 없거나 이름이 다른 경우)
gu_error = df_main[
    (df_main['CGG_NM'] != df_main['검증_CGG_NM']) & 
    (df_main['검증_CGG_NM'].notnull()) # 매핑 자체가 안된 건 별도 확인 필요
]

print(f"🛑 CGG_NM 불일치 건수: {len(gu_error)}건")
if len(gu_error) > 0:
    print("\n[불일치 예시]")
    display(gu_error[['CGG_CD', 'CGG_NM', '검증_CGG_NM']].head())

# ==========================================
# 🔍 2단계: 법정동 검증 (Dong Validation)
# ==========================================

# 정답지에서 {(CGG_CD + STDG_CD): STDG_NM} 딕셔너리 생성
# (이유: STDG_CD는 구마다 중복될 수 있으므로, CGG_CD와 결합해서 Key로 써야 안전함)
df_ref['key'] = df_ref['CGG_CD'] + df_ref['STDG_CD']
df_main['key'] = df_main['CGG_CD'] + df_main['STDG_CD']

dong_mapper = dict(zip(df_ref['key'], df_ref['STDG_NM']))

# 매핑 및 비교
df_main['검증_STDG_NM'] = df_main['key'].map(dong_mapper)

dong_error = df_main[
    (df_main['STDG_NM'] != df_main['검증_STDG_NM']) &
    (df_main['검증_STDG_NM'].notnull())
]

print(f"🛑 STDG_NM 불일치 건수: {len(dong_error)}건")
if len(dong_error) > 0:
    print("\n[불일치 예시]")
    display(dong_error[['CGG_CD', 'STDG_CD', 'STDG_NM', '검증_STDG_NM']].head(100))

# ==========================================
# 🔍 3단계: 코드 자체가 정답지에 없는 경우
# ==========================================
unknown_code = df_main[df_main['검증_CGG_NM'].isnull() | df_main['검증_STDG_NM'].isnull()]

if len(unknown_code) > 0:
    print(f"\n⚠️ 정답지에 존재하지 않는 코드가 발견됨: {len(unknown_code)}건")
    display(unknown_code[['CGG_CD', 'STDG_CD']].drop_duplicates())
else:
    print("\n✅ 모든 코드가 정답지에 정상적으로 존재합니다.")

C:\Users\gorhk\AppData\Local\Temp\ipykernel_93256\2348783595.py:5: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_main = pd.read_csv('data/real_estate_total_edit.csv', encoding='euc-kr')


🛑 CGG_NM 불일치 건수: 0건
🛑 STDG_NM 불일치 건수: 21건

[불일치 예시]


,CGG_CD,STDG_CD,STDG_NM,검증_STDG_NM
34840,11290,10700,청량리동,동소문동4가
37890,11590,10100,봉천동,노량진동
161601,11170,10100,방배동,후암동
169866,11305,10300,화곡동,수유동
170513,11470,10300,가리봉동,신월동
171554,11380,10100,중곡동,수색동
185005,11650,10200,상도동,양재동
198162,11290,10700,청량리동,동소문동4가
207391,11290,10700,청량리동,동소문동4가
350143,11290,10700,청량리동,동소문동4가



⚠️ 정답지에 존재하지 않는 코드가 발견됨: 3건


,CGG_CD,STDG_CD
606401,11620,10700
829987,26230,10100


In [7]:
df_ref = pd.read_csv('data/법정동합본.csv', encoding='euc-kr')
df_ref

,자치구코드,자치구명,법정동코드,법정동명
0,11110,종로구,10100,청운동
1,11110,종로구,10200,신교동
2,11110,종로구,10300,궁정동
3,11110,종로구,10400,효자동
4,11110,종로구,10500,창성동
...,...,...,...,...
462,11740,강동구,10600,둔촌동
463,11740,강동구,10700,암사동
464,11740,강동구,10800,성내동
465,11740,강동구,10900,천호동
